In [1]:
%display latex
from dalgebra import *

### Small example on how to use `dalgebra`

In this notebook I portrait how to use the package `dalgebra` to compute differential resultants and, as we will see, sometimes solve some differential systems.

The system we are goind to study in this notebook is the following:
$$\left\{\begin{array}{ll}
    xu+x2u′′−(1−x)v & =0,\\
    v′−v′′+u′ & =0
\end{array}\right.$$
where $x$ is the standard variable (i.e., $x′=1$) and the variables $u$ and $v$ are differential variables that can be considered as functions on $x$ (i.e., $u=u(x)$ and $v=v(x)$).

##### 1. Building the main ring for representing the system

Using `dalgebra` we can easily create the ring of differential polynomials required to represent the system with the following syntax:

In [2]:
R.<u,v> = DifferentialPolynomialRing(QQ[x])
x = R.base().gens()[0]
R

Ring of operator polynomials in (u, v) over Differential Ring [[Univariate Polynomial Ring in x over Rational Field], (d/dx,)]

Where now the variable `R` represent the ring of differential polynomials on the variables $u$ and $v$ with coefficients in the polynomial ring $Q[x]$.

#### 2. Building the differential system

We can now use the variables `u` and `v` to create the variables $u(x)$
and $v(x)$ and all its derivatives. For doing so, we use the syntax `u[i]` to create the $i$-th derivative of the function $u(x)$.

In [3]:
f1 = x*u[0] + x^2*u[2] - (1-x)*v[0]
f1

x^2*u_2 + x*u_0 + (x - 1)*v_0

In [4]:
f2 = v[1]-v[2]+u[1]
f2

u_1 - v_2 + v_1

Now we need to create the system itself. This is done with the class `DifferentialSystem`:

In [5]:
DifferentialSystem([f1,f2])

System over [Ring of operator polynomials in (u, v) over Differential Ring [[Univariate Polynomial Ring in x over Rational Field], (d/dx,)]] with variables [(u_*, v_*)]:
{
	x^2*u_2 + x*u_0 + (x - 1)*v_0 == 0
	u_1 - v_2 + v_1 == 0
}

However, this is not the most useful for solving the system, since the elimination by resultant requires to fix one of the variables as main variables. This is done with the optional parameter variables of the constructor of `DifferentialSystem`:

In [6]:
system = DifferentialSystem([f1,f2], variables=[u])
system

System over [Ring of operator polynomials in (u, v) over Differential Ring [[Univariate Polynomial Ring in x over Rational Field], (d/dx,)]] with variables [(u_*,)]:
{
	x^2*u_2 + x*u_0 + (x - 1)*v_0 == 0
	u_1 - v_2 + v_1 == 0
}

#### 3. Extending the system by derivation

The key to apply algebraic techniques to a differential system is to extend the system by derivations. In terms of differential ideals, we are taking other elements in the differential ideal to have extra information for solving the system.

In the package `dalgebra` we offer the methods `extend_by_derivation` or `build_sp1` to do so. We need to provide a list of integers saying the extension for each equation of the system.

The output for these methods is a new `DifferentialSystem` with more equations.

In [7]:
system.extend_by_derivation([1,3])

System over [Ring of operator polynomials in (u, v) over Differential Ring [[Univariate Polynomial Ring in x over Rational Field], (d/dx,)]] with variables [(u_*,)]:
{
	x^2*u_2 + x*u_0 + (x - 1)*v_0 == 0
	x^2*u_3 + 2*x*u_2 + x*u_1 + u_0 + (x - 1)*v_1 + v_0 == 0
	u_1 - v_2 + v_1 == 0
	u_2 - v_3 + v_2 == 0
	u_3 - v_4 + v_3 == 0
	u_4 - v_5 + v_4 == 0
}

#### 4. Checking the amount of algebraic elements

Once we have an extended system, we need to get the condition **SP2** from the paper in order to compute an algebraic resultant to the system. This condition requires to look each of the derivatives of $u(x)$ and $v(x)$
as independent algebraic variables, count them and check we have the appropriate number of variables.

This can be checked in the code with the methods:

* `algebraic_equations`: return the equations of the system but considered as simply polynomials (with no differential structured attached). The polynomials are organized with the main variables (given when creating the system) in the top level, and all other variables in a lower level.
* `algebraic_variables`: return the variables of the system (given when we created the system) that show up in the equations of the system.
* `is_sp2`: check that the number of equations is the number of algebraic variables plus one.

In [8]:
system

System over [Ring of operator polynomials in (u, v) over Differential Ring [[Univariate Polynomial Ring in x over Rational Field], (d/dx,)]] with variables [(u_*,)]:
{
	x^2*u_2 + x*u_0 + (x - 1)*v_0 == 0
	u_1 - v_2 + v_1 == 0
}

In [9]:
system.algebraic_variables()

(u_0, u_1, u_2)

In [10]:
system.extend_by_derivation([1,2]).algebraic_variables()

(u_0, u_1, u_2, u_3)

In [11]:
system.extend_by_derivation([1,2]).size()

5

In [12]:
system.extend_by_derivation([1,2]).is_sp2()

True

#### 5. Computing the differential resultant

Putting all the previous methods together, we can then use a resultant formula to obtain the differential resultant of the system. The method differential_resultant work as follows:

1. Tries to extend the system to satisfy condition **SP2**. The limit for extension is given by a bound in the algorithm ($10$ by default).
2. Computes the algebraic resultant of the extended system. This can be done right now using the code for *Macaulay resultant* from Sage.

In [13]:
system.diff_resultant(alg_res="macaulay")

(-x^3)*v_4 + (x^3 - x^2)*v_3 + x*v_1 - v_0

#### 6. Solving the system

Once we have the differential resultant, we had eliminated the variable $u$ and obtained a linear differential equation for $v(x)$ in the same differential ideal. We can solve this equation with any tool and then plug it to the original system to obtain a differential equation for $u(x)$ and solve it too.

To illustrate how we could do this, I use the package `dd_functions` to represent the solutions to linear differential equations.

In [14]:
from ajpastor.dd_functions import *
DFiniteAB = ParametrizedDDRing(DFinite, ['a','b'])
a = DFiniteAB.parameter('a'); b = DFiniteAB.parameter('b')
sol_v = DFiniteAB.element([-1, x, 0, x^3-x^2, -x^3],[0,a,b])
sol_v

(4:4:15)DD-Function in (DD-Ring over (Univariate Polynomial Ring in x over Rational Field) with parameters (b, a))

And now we use the second equation from our original system to create the element $u(x)$:

In [15]:
sol_u = DFiniteAB.element([0,1], inhomogeneous=sol_v.derivative(times=2) - sol_v.derivative())
sol_u

(8:8:33)DD-Function in (DD-Ring over (Univariate Polynomial Ring in x over Rational Field) with parameters (b, a))

We can see the conditions on the parameters $a$ and $b$ to these two be also solutions of the first equation of the system:

In [16]:
system.equation(0)(u=sol_u, v=sol_v)

TypeError: Impossible to compute a pushout: generators not recognized found.
	- [x]

Any solution with $a=0$ is a solution of the whole differential system:

In [17]:
final_v = DFiniteAB.element([-1, x, 0, x^3-x^2, -x^3],[0,0,b])
final_u = DFiniteAB.element([0,1], inhomogeneous=final_v.derivative(times=2) - final_v.derivative())

all(equation(u=final_u, v=final_v) == 0 for equation in system.equations())

TypeError: Impossible to compute a pushout: generators not recognized found.
	- [x]